In [12]:
 # !python -m "pyalgotrade.tools.quandl" --source-code="WIKI" --table-code="aapl" --from-year=2000 --to-year=2000 --storage=. --force-download --frequency=daily
import yfinance as yf
from datetime import datetime

# ticker = '^gspc'
ticker = 'tsla'

start_date = datetime(2020, 1, 1)
end_date = datetime(2022, 9, 30)
data = yf.download(ticker, start = start_date,end = end_date, auto_adjust = False)
data.to_csv('data.csv')

[*********************100%***********************]  1 of 1 completed


In [2]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,320.529999,322.130005,320.149994,321.859985,308.234406,57077300
2020-01-02,323.540009,324.890015,322.529999,324.869995,311.117035,59151200
2020-01-03,321.160004,323.640015,321.100006,322.410004,308.761200,77709700
2020-01-06,320.489990,323.730011,320.359985,323.640015,309.939148,55653900
2020-01-07,323.019989,323.540009,322.239990,322.730011,309.067688,40496400
...,...,...,...,...,...,...
2022-09-23,370.579987,370.619995,363.290009,367.950012,367.950012,122210100
2022-09-26,366.410004,370.209991,363.029999,364.309998,364.309998,92581200
2022-09-27,368.019989,370.399994,360.869995,363.380005,363.380005,108294100


In [13]:
# Interval = Day K

# from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import ma
from pyalgotrade.technical import rsi
from pyalgotrade.technical import stoch
from pyalgotrade.bar import Frequency


number_of_shares = 1

def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 100000) # initial equity
        
        # We'll use adjusted close values instead of regular close values.
        # self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), smaPeriod)
        self.__rsi = rsi.RSI(feed[instrument].getCloseDataSeries(), 6)
        self.__kdj = stoch.StochasticOscillator(feed[instrument], 9, dSMAPeriod=3)
        self.D = self.__kdj.getD()
        self.__position = None
        self.__instrument = instrument

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-1], 2),safe_round(self.__rsi[-1], 2)))
        global buy, total_buy
        buy = execInfo.getPrice() * number_of_shares
        total_buy += buy

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-1], 2),safe_round(self.__rsi[-1], 2)))
        self.__position = None
        sell = execInfo.getPrice() * number_of_shares
        global return_per_share, total_sell
        total_sell += sell
        return_per_share = total_sell - total_buy

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        # bar = bars[self.__instrument]
        
        # ALL 30 70
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #     safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        # elif safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and not self.__position.exitActive() and \
        # safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #     self.__position.exitMarket()

        # ALL 30 70
        if self.__position is None:
            if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
            safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        elif safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and not self.__position.exitActive() and \
        safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
            self.__position.exitMarket()

        # ALL 20 80
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #     safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        # elif safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and not self.__position.exitActive() and \
        # safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #     self.__position.exitMarket()

        # ALL 20 80 
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #     safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        # elif safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and not self.__position.exitActive() and \
        # safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #     self.__position.exitMarket()

        # Just < 30 and > 70
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        # elif safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and not self.__position.exitActive():
        #     self.__position.exitMarket()

        # Just < 20 and > 80
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        # elif safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and not self.__position.exitActive():
        #     self.__position.exitMarket()

        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)


def run_strategy(smaPeriod):
    global total_sell, total_buy
    total_buy = 0
    total_sell = 0
    # Load the bar feed from the CSV file
    # feed = quandlfeed.Feed()
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE)
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV(ticker, "data.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, ticker, smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())
    print('Return = ', return_per_share)

run_strategy(15)


2020-03-17 00:00:00 strategy [INFO] BUY at $29.33 9.78 12.73
2020-04-16 00:00:00 strategy [INFO] SELL at $47.80 95.11 88.37
2021-02-22 00:00:00 strategy [INFO] BUY at $254.21 2.87 13.17
2021-06-29 00:00:00 strategy [INFO] SELL at $228.22 82.49 73.02
2021-12-21 00:00:00 strategy [INFO] BUY at $305.62 29.72 39.8
2022-03-23 00:00:00 strategy [INFO] SELL at $326.65 85.39 82.01
2022-05-02 00:00:00 strategy [INFO] BUY at $286.92 30.03 38.48
2022-07-29 00:00:00 strategy [INFO] SELL at $280.70 98.09 82.65
2022-09-01 00:00:00 strategy [INFO] BUY at $272.58 29.36 27.95


Final portfolio value: $100002.90
Return =  7.265346527099609
